In [476]:
from sympy import *
import numpy as np
import scipy as sp
import scipy.linalg
import time

### Get a 16-atom Si configuration as a reference.

In [537]:
# create supercell
def get_supercell(unit_cell, dim, unit_pos):
    # initialize position list
    positions = []

    # define bravais lattice vectors
    vec1 = np.array(unit_cell[0])
    vec2 = np.array(unit_cell[1])
    vec3 = np.array(unit_cell[2])

    # append positions of atoms in supercell
    for m in range(dim):
        for n in range(dim):
            for p in range(dim):
                for q in range(len(unit_pos)):
                    positions.append([unit_pos[q][0], \
                        list(np.array(unit_pos[q][1]) +\
                         m*vec1 + n*vec2 + p*vec3)])
                    
    # get supercell dimensions
    supercell = list(np.array(unit_cell)*dim)
                    
    return positions, supercell

# perturb the positions in a supercell
def perturb_struc(positions, pert_size):
    # loop through positions and add a random perturbation
    for n in range(len(positions)):
        for m in range(3):
            # get current coordinate
            coord_curr = positions[n][1][m]

            # get perturbation by drawing from uniform
            pert = np.random.uniform(-pert_size, pert_size)

            # perturb the coordinate
            positions[n][1][m] += pert
            
    return positions

# put supercell positions and cell parameters in QE friendly format
# based on Boris K's AP275 code
def get_position_txt(positions, supercell):
    
    # write atomic positions
    postxt = ''
    postxt += 'ATOMIC_POSITIONS {angstrom}'
    for pos in positions:
        postxt += '\n {} {:1.5f} {:1.5f} {:1.5f}'.format(pos[0], *pos[1])
        
    # write cell parameters
    celltxt = ''
    celltxt += 'CELL_PARAMETERS {angstrom}'
    for vector in supercell:
        celltxt += '\n {:1.5f} {:1.5f} {:1.5f}'.format(*vector)
    return postxt, celltxt

# get perturbed positions
def get_perturbed_pos(unit_cell, dim, unit_pos, pert_size):
    # get perturbed structure
    positions, supercell = get_supercell(unit_cell, dim, unit_pos)
    positions = perturb_struc(positions, pert_size)
    pos, cell = get_position_txt(positions, supercell)
    
    #get position array
    pos_array = [positions[n][1] for n in range(len(positions))]

    return pos, cell, pos_array, supercell, positions

In [573]:
# Si crystal structure
alat = 5.431
pert_size = 0.05 * alat # size of initial perturbation
dim = 1
unit_cell = [[0.0, alat/2, alat/2], [alat/2, 0.0, alat/2], \
                [alat/2, alat/2, 0.0]] # fcc primitive cell
unit_pos = [['Si',[0,0,0]],['Si',[alat/4, alat/4, alat/4]]]
brav_mat = np.array([[0.0, alat/2, alat/2], [alat/2, 0.0, alat/2], \
            [alat/2, alat/2, 0.0]])*dim
brav_inv = np.linalg.inv(brav_mat)

pos_text, cell, pos, supercell, pos_label = \
            get_perturbed_pos(unit_cell, dim, unit_pos, pert_size)

cutoff = 4.5
    
# bravais vectors
vec1 = brav_mat[:,0].reshape(3,1)
vec2 = brav_mat[:,1].reshape(3,1)
vec3 = brav_mat[:,2].reshape(3,1)
    
pos

[[0.08594930072026763, 0.020590366090149137, 0.13849312295838323],
 [1.21494124548106, 1.3108824132479175, 1.284410446090832]]

We'll use 3xN arrays (where N is the number of atoms in the supercell) as the standard format for global configurations.

In [574]:
global_pos = np.array(pos).transpose()
global_pos

array([[0.0859493 , 1.21494125],
       [0.02059037, 1.31088241],
       [0.13849312, 1.28441045]])

In [575]:
global_pos.shape

(3, 2)

### Create local configurations.

In [581]:
# for a given vector, get images within cutoff radius
# ref_vec, vec, vec1, vec2, vec3 assumed to be column vectors
def get_images(ref_vec, vec, brav_mat, brav_inv, vec1, vec2, vec3, cutoff):
    # get bravais coefficients of position vector relative to reference atom
    coeff = np.matmul(brav_inv, vec-ref_vec)
    
    # get bravais coefficients for atoms within one super-super-super-cell
    coeffs = [[],[],[]]
    for n in range(3):
        coeffs[n].append(coeff[n][0])
        coeffs[n].append(coeff[n][0]-1)
        coeffs[n].append(coeff[n][0]+1)
        coeffs[n].append(coeff[n][0]-2)
        coeffs[n].append(coeff[n][0]+2)
        coeffs[n].append(coeff[n][0]-3)
        coeffs[n].append(coeff[n][0]+3)
        coeffs[n].append(coeff[n][0]-4)
        coeffs[n].append(coeff[n][0]+4)

    # get vectors within cutoff
    vecs = []
    for m in range(len(coeffs[0])):
        for n in range(len(coeffs[1])):
            for p in range(len(coeffs[2])):
                vec_curr = coeffs[0][m]*vec1 + coeffs[1][n]*vec2 + coeffs[2][p]*vec3
                
                dist = np.linalg.norm(vec_curr)

                if dist < cutoff:
                    vecs.append(vec_curr+ref_vec)
                    
    return vecs

In [582]:
# with global configuration as input, get local configuration of a specified atom
# first column contains coordinates and label of reference atom
# remaining columns contain coordinates and labels of atoms within cutoff region (including images)
def get_loc(global_pos, atom, cutoff,brav_mat, brav_inv, vec1, vec2, vec3):
    # get Cartesian coordinates of reference atom
    ref_vec = global_pos[:,atom].reshape(3,1)

    # set number of atoms
    noa = global_pos.shape[1]

    # initialize local environment by appending atom label
    loc = np.vstack((ref_vec,atom))

    # add images within cutoff radius
    for n in range(noa):
        if n != atom:
            vec = global_pos[:,n].reshape(3,1)
            images = get_images(ref_vec, vec, brav_mat, brav_inv, vec1, vec2, vec3, cutoff)

            # append image coordinates and labels
            for m in range(len(images)):
                vec_lab = np.vstack((images[m],n))
                loc = np.hstack((loc, vec_lab))
                
    return loc

# with global configuration as input, get local configurations of all atoms
def get_locs(global_pos, cutoff,brav_mat, brav_inv, vec1, vec2, vec3):
    noa = global_pos.shape[1]
    locs = []
    for n in range(noa):
        loc_curr = get_loc(global_pos, n, cutoff,brav_mat, brav_inv, vec1, vec2, vec3)
        locs.append(loc_curr)
    return locs      

In [583]:
locs = get_locs(global_pos, cutoff,brav_mat, brav_inv, vec1, vec2, vec3)
print(locs)

[array([[ 0.0859493 ,  1.21494125, -1.50055875, -1.50055875,  1.21494125,
         1.21494125,  1.21494125,  3.93044125,  3.93044125,  1.21494125,
        -1.50055875, -1.50055875],
       [ 0.02059037,  1.31088241, -1.40461759,  1.31088241,  4.02638241,
        -1.40461759, -1.40461759,  1.31088241, -1.40461759, -4.12011759,
         1.31088241,  4.02638241],
       [ 0.13849312,  1.28441045,  1.28441045, -1.43108955, -1.43108955,
         3.99991045, -1.43108955, -1.43108955,  1.28441045,  1.28441045,
         3.99991045,  1.28441045],
       [ 0.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ]]), array([[ 1.21494125,  0.0859493 ,  2.8014493 ,  0.0859493 ,  2.8014493 ,
         0.0859493 ,  2.8014493 ,  2.8014493 ,  0.0859493 , -2.6295507 ,
        -2.6295507 ,  0.0859493 ],
       [ 1.31088241,  0.02059037,  2.73609037,  2.73609037,  0.02059037,
        -2.69490963

### Calculate two-body kernel.

In [24]:
# get two-body kernel between two local environments
def get_tb_base(rho1, rho2, sig, ls):
    # get reference vectors
    ref1 = rho1[0:3,0].reshape(3,1)
    ref2 = rho2[0:3,0].reshape(3,1)
    
    # loop over atoms (skipping over reference atom)
    tot_kern = 0
    for m in range(1,rho1.shape[1]):
        vec1 = rho1[0:3,m].reshape(3,1)
        
        for n in range(1,rho2.shape[1]):
            vec2 = rho2[0:3,n].reshape(3,1)
            
            # calculate covariance for given pair
            dist1 = np.linalg.norm(vec1-ref1)
            dist2 = np.linalg.norm(vec2-ref2)
            kern = sig**2*np.exp(-(dist1-dist2)**2/(2*ls**2))
            tot_kern += kern
            
    return tot_kern

In [128]:
comp_no_1 = 8
# comp_no_2 = 4
for n in range(16):
    comp_no_2 = n
    tot_kern = get_tb_base(locs[comp_no_1],locs[comp_no_2],1,0.01)/1
#         ((locs[comp_no_1].shape[1]-1)*(locs[comp_no_2].shape[1]-1))
    print(tot_kern)

13.042559225600275
12.78505611642455
8.77436392829975
11.255097467995327
13.72631313929308
11.078188494313935
9.712778768615099
11.941311644065575
30.19236772920608
12.539462123111377
13.391983263233444
12.281924977561854
10.036103110705971
14.394386129876004
13.8561940293867
12.183178771968139


### Calculate full two-body kernel.

In [231]:
locs

[array([[ 0.16913275,  1.30691811, -1.40858189, -1.40858189,  1.30691811],
        [ 0.08296065,  1.21610812, -1.49939188,  1.21610812, -1.49939188],
        [-0.02519314,  1.34500012,  1.34500012, -1.37049988, -1.37049988],
        [ 0.        ,  1.        ,  1.        ,  1.        ,  1.        ]]),
 array([[ 1.30691811,  0.16913275,  2.88463275,  2.88463275,  0.16913275],
        [ 1.21610812,  0.08296065,  2.79846065,  0.08296065,  2.79846065],
        [ 1.34500012, -0.02519314, -0.02519314,  2.69030686,  2.69030686],
        [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ]])]

In [232]:
locs[0]

array([[ 0.16913275,  1.30691811, -1.40858189, -1.40858189,  1.30691811],
       [ 0.08296065,  1.21610812, -1.49939188,  1.21610812, -1.49939188],
       [-0.02519314,  1.34500012,  1.34500012, -1.37049988, -1.37049988],
       [ 0.        ,  1.        ,  1.        ,  1.        ,  1.        ]])

In [233]:
locs[1]

array([[ 1.30691811,  0.16913275,  2.88463275,  2.88463275,  0.16913275],
       [ 1.21610812,  0.08296065,  2.79846065,  0.08296065,  2.79846065],
       [ 1.34500012, -0.02519314, -0.02519314,  2.69030686,  2.69030686],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [362]:
# get two body kernel (no derivatives)
def derv0(vec1, ref1, vec2, ref2, sig, ls):
    # calculate covariance for current pair of atoms
    dist1 = np.linalg.norm(vec1-ref1)
    dist2 = np.linalg.norm(vec2-ref2)
    kern = sig**2*np.exp(-(dist1-dist2)**2/(2*ls**2))
    return kern

In [363]:
# get two body kernel (one derivative)
def derv1(en_vec, en_ref, force_vec, force_ref,\
          force_atom, ref_atom, env_atom, force_comp,\
          sig, ls):       
    # check if force atom equals reference atom or environment atom
    if int(force_atom)==int(ref_atom):
        coord_diff = force_ref[force_comp][0]-force_vec[force_comp][0]
    elif int(force_atom)==int(env_atom):
        coord_diff = force_vec[force_comp][0]-force_ref[force_comp][0]
    else:
        return 0

    # calculate covariance for current pair of atoms
    en_dist = np.linalg.norm(en_vec-en_ref)
    force_dist = np.linalg.norm(force_vec-force_ref)
    base = sig**2*np.exp(-(en_dist-force_dist)**2/(2*ls**2))
    kern = base*coord_diff*(force_dist-en_dist)/(force_dist*ls**2)
    return kern

In [364]:
# get two body kernel (two derivatives)
def derv2(fa1, ra1, ea1, fc1, \
          fa2, ra2, ea2, fc2, \
          vec1, ref1, vec2, ref2,\
          sig, ls): 

    # check if force atom equals reference atom or environment atom
    if int(fa1)==int(ra1):
        coord1 = ref1[fc1][0]-vec1[fc1][0]
    elif int(fa1)==int(ea1):
        coord1 = vec1[fc1][0]-ref1[fc1][0]
    else:
        return 0

    # check if force atom equals reference atom or environment atom
    if int(fa2)==int(ra2):
        coord2 = ref2[fc2][0]-vec2[fc2][0]
    elif int(fa2)==int(ea2):
        coord2 = vec2[fc2][0]-ref2[fc2][0]
    else:
        return 0

    # calculate covariance for current pair of atoms
    dist1 = np.linalg.norm(vec1-ref1)
    dist2 = np.linalg.norm(vec2-ref2)
    base = sig**2*np.exp(-(dist1-dist2)**2/(2*ls**2))
    doub_fac = -ls**2+(dist1-dist2)**2
    kern = -base*coord1*coord2*doub_fac/(dist1*dist2*ls**4)
    
    return kern

In [423]:
# get kernel when two global energies are compared
def kern_ee(rho1, rho2, sig, ls):
    tot_kern = 0
    
    # get reference vectors
    ref1 = rho1[0:3,0].reshape(3,1)
    ref2 = rho2[0:3,0].reshape(3,1)
    
    # loop over atoms in environment 1
    for m in range(1,rho1.shape[1]):
        vec1 = rho1[0:3,m].reshape(3,1)

        # loop over atoms in environment 2
        for n in range(1,rho2.shape[1]):
            vec2 = rho2[0:3,n].reshape(3,1)
            kern = derv0(vec1, ref1, vec2, ref2, sig, ls)
            tot_kern += kern     
    return tot_kern

In [424]:
# get kernel when an energy is compared to a force
def kern_ef(rho1, rho2, d1, d2, sig, ls):
    tot_kern=0
    # define energy environment and force environment
    if d1==0:
        en_ref = rho1[0:3,0].reshape(3,1)
        force_ref = rho2[0:3,0].reshape(3,1)
        en_env = rho1
        force_env = rho2
        force_atom = d2[0]
        ref_atom = force_env[3,0]
        force_comp = d2[1]
    if d2==0:
        en_ref = rho2[0:3,0].reshape(3,1)
        force_ref = rho1[0:3,0].reshape(3,1)
        en_env = rho2
        force_env = rho1
        force_atom = d1[0]
        ref_atom = force_env[3,0]
        force_comp = d1[1]

    # loop over atoms in energy environment
    for m in range(1,en_env.shape[1]):
        en_vec = en_env[0:3,m].reshape(3,1)

        # loop over atoms in force environment
        for n in range(1,force_env.shape[1]):
            force_vec = force_env[0:3,n].reshape(3,1)
            env_atom = force_env[3,n]
            kern = derv1(en_vec, en_ref, force_vec, force_ref,\
                          force_atom, ref_atom, env_atom, force_comp,\
                          sig, ls)         
            tot_kern+=kern
    return tot_kern

In [378]:
# get kernel when two forces are compared
def kern_ff(rho1, rho2, d1, d2, sig, ls):
    tot_kern=0
    # set force atoms and components
    fa1 = d1[0]
    fc1 = d1[1]
    fa2 = d2[0]
    fc2 = d2[1]
    
    # set reference vectors and atoms
    ref1 = rho1[0:3,0].reshape(3,1)
    ref2 = rho2[0:3,0].reshape(3,1)
    ra1 = rho1[3,0]
    ra2 = rho2[3,0]

    # loop over atoms in force environment 1
    for m in range(1,rho1.shape[1]):
        vec1 = rho1[0:3,m].reshape(3,1)
        ea1 = rho1[3,m]

        # loop over atoms in force environment 2
        for n in range(1,rho2.shape[1]):
            vec2 = rho2[0:3,n].reshape(3,1)
            ea2 = rho2[3,n]
            
            kern = derv2(fa1, ra1, ea1, fc1, \
                      fa2, ra2, ea2, fc2, \
                      vec1, ref1, vec2, ref2,\
                      sig, ls)
            tot_kern+= kern
    return tot_kern

In [368]:
def get_kern(rho1, rho2, d1, d2, sig, ls):
    # first case: comparing two global energies
    if d1==0 and d2==0:
        tot_kern = kern_ee(rho1, rho2, sig, ls)

    # second case: compare global energy to force
    # first element of d is the force atom
    # second element is the component (0=x,1=y,2=z)
    if (d1==0 and d2!=0) or (d2==0 and d1!=0):
        tot_kern = kern_ef(rho1, rho2, d1, d2, sig, ls)

    # third and final case: compare two force components
    if d1!=0 and d2!=0:
        tot_kern = kern_ff(rho1, rho2, d1, d2, sig, ls)
                
    return tot_kern

In [345]:
tot_kern

-2.948479810389541

In [299]:
locs

[array([[ 0.16913275,  1.30691811, -1.40858189, -1.40858189,  1.30691811],
        [ 0.08296065,  1.21610812, -1.49939188,  1.21610812, -1.49939188],
        [-0.02519314,  1.34500012,  1.34500012, -1.37049988, -1.37049988],
        [ 0.        ,  1.        ,  1.        ,  1.        ,  1.        ]]),
 array([[ 1.30691811,  0.16913275,  2.88463275,  2.88463275,  0.16913275],
        [ 1.21610812,  0.08296065,  2.79846065,  0.08296065,  2.79846065],
        [ 1.34500012, -0.02519314, -0.02519314,  2.69030686,  2.69030686],
        [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ]])]

### Test kernel derivatives by comparing to Mathematica evaluations.

In [425]:
# test derv0
sig=1
ls=1
vec1 = np.array([1,2,3]).reshape(3,1)
ref1 = np.array([4,5,6]).reshape(3,1)
vec2 = np.array([7,8,9]).reshape(3,1)
ref2 = np.array([2,4,6]).reshape(3,1)
derv0(vec1, ref1, vec2, ref2, sig, ls)

0.1724489793313035

In [426]:
# test derv1
sig=1
ls=1
en_vec = np.array([1,2,3]).reshape(3,1)
en_ref = np.array([4,5,6]).reshape(3,1)
force_vec = np.array([7,8,9]).reshape(3,1)
force_ref = np.array([2,4,6]).reshape(3,1)
force_atom = 10
ref_atom = 10
env_atom = 11
force_comp = 2

derv1(en_vec, en_ref, force_vec, force_ref,\
          force_atom, ref_atom, env_atom, force_comp,\
          sig, ls)

-0.1371761325709999

In [427]:
# test derv2
fa1 = 2
ra1 = 2
ea1 = 1
fc1 = 0

fa2 = 2
ra2 = 2
ea2 = 1
fc2 = 1

vec1 = np.array([1,2,3]).reshape(3,1)
ref1 = np.array([4,5,6]).reshape(3,1)
vec2 = np.array([7,8,9]).reshape(3,1)
ref2 = np.array([2,4,6]).reshape(3,1)

sig=1
ls=1

derv2(fa1, ra1, ea1, fc1, \
          fa2, ra2, ea2, fc2, \
          vec1, ref1, vec2, ref2,\
          sig, ls)

0.1416661571066722

### Test kernel on a simple configuration by comparing to Mathematica evaluations.

In [480]:
# create fictitious 3-atom system
x = 1
y = 2
z = 3

loc1 = np.array([[x,0,0],[0,y,0],[0,0,z],[0,1,2]])
loc2 = np.array([[0,x,0],[y,0,0],[0,0,z],[1,0,2]])
loc3 = np.array([[0,x,0],[0,0,y],[z,0,0],[2,0,1]])
S = [loc1, loc2, loc3] # snapshot

In [429]:
# test kernel function
rho1 = loc1
rho2 = loc2
d1 = [2,0]
d2 = 0
sig = 1
ls = 1
get_kern(rho1, rho2, d1, d2, sig, ls)

-0.06367459403901385

In [420]:
rho1 = loc1
rho2 = loc2
d1 = [2,0]
d2 = [1,1]
sig = 1
ls = 1
get_kern(rho1, rho2, d1, d2, sig, ls)

-0.1539356948466952

In [421]:
rho1 = loc1
rho2 = loc2
d1 = 0
d2 = 0
sig = 1
ls = 1
get_kern(rho1, rho2, d1, d2, sig, ls)

2.9491409811449683

### Generalize to global configurations.

In [468]:
# given a dataset, get the size of the kernel matrix
def get_size(D):
    size = 0
    for n in range(len(D)):
        # for each snapshot, account for one global energy and 3N force components
        size+=1+3*len(D[n])
    return size

In [509]:
# get covariance between two MD snapshots
def snap_kern(S1, S2, d1, d2, sig, ls):
    tot_kern = 0
    # loop through atoms of first snapshot
    for m in range(len(S1)):
        rho1 = S1[m]
        
        # loop through atoms of second snapshot
        for n in range(len(S2)):
            rho2 = S2[n]
            kern = get_kern(rho1, rho2, d1, d2, sig, ls)
            tot_kern+=kern
    return tot_kern

In [567]:
# dataset D is a list of MD snapshots S containing a list of local environments P
D = [locs] # dataset consisting of a single snapshot

# set hyperparameters
sig = 1
ls = 1

size = get_size(D)
K = np.zeros([size, size])

# start the clock
tic = time.time()

# set first snapshot
row = 0
for p1 in range(len(D)):
    snap1 = D[p1]
    
    # loop through energy+force components
    for p2 in range(1+3*len(snap1)):
        # global energy
        if p2==0:
            d1=0
          
        # force component
        else:
            at1 = int(floor((p2-1)/3)) # atom
            fc1 = (p2-1)%3 # force component (0=x,1=y,2=z)
            d1 = [at1, fc1]
        

        # set second snapshot
        col = 0
        for p3 in range(len(D)):
            snap2 = D[p3]

            # loop through atoms in second snapshot
            for p4 in range(1+3*len(snap2)):
                # global energy
                if p4==0:
                    d2=0
                    
                # force component
                else:
                    at2 = int(floor((p4-1)/3))
                    fc2 = (p4-1)%3
                    d2 = [at2, fc2]


                kern = snap_kern(snap1, snap2, d1, d2, sig, ls)
                K[row,col]=kern
                
                col+=1
                print(col)
                
        row+=1
             
toc = time.time()
tot_time = toc-tic
print(tot_time)
    
test = np.linalg.inv(K)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1

In [526]:
# create fictitious 3-atom system
x = 1
y = 2
z = 3

loc1 = np.array([[x,0,0,3],[0,y,0,1],[0,0,z,4],[0,1,2,3]])
loc2 = np.array([[0,x,0,3],[y,0,0,1],[0,0,z,4],[1,0,2,3]])
loc3 = np.array([[0,x,0,3],[0,0,y,1],[z,0,0,4],[2,0,1,3]])
loc4 = np.array([[3,0,x,0],[1,0,0,y],[4,z,0,0],[3,2,0,1]])
S = [loc1, loc2, loc3, loc4] # snapshot
D = [S]

In [568]:
len(locs)

16

In [570]:
locs[0].shape

(4, 24)

In [557]:
col

14

In [558]:
size

49

In [559]:
49*49

2401

In [560]:
locs[0]

array([[-1.63516593e-01,  1.41084937e+00,  2.90139257e+00,
        -2.52960743e+00, -1.11260744e+00, -2.71534896e+00,
        -1.27226227e+00,  2.24434344e-01,  1.91843536e-01,
         1.91843536e-01,  1.40520469e+00, -2.51497117e+00,
        -2.95890964e+00,  2.47209036e+00],
       [ 1.17619276e-02,  1.43829244e+00,  2.56213801e+00,
        -2.86886199e+00, -1.21516007e+00,  2.36729614e-01,
         1.15476933e+00,  2.90943977e+00,  2.74482922e+00,
        -2.68617078e+00, -1.37493412e+00,  1.98360566e-02,
         2.83353911e+00, -2.59746089e+00],
       [-1.53403947e-01,  1.34052804e+00, -1.40055782e-01,
        -1.40055782e-01,  1.57062603e+00, -2.57912894e+00,
        -1.52128545e+00, -2.48784178e+00,  2.68416658e+00,
        -2.74683342e+00, -1.38996649e+00,  2.63257179e+00,
        -3.76179767e-02, -3.76179767e-02],
       [ 0.00000000e+00,  1.00000000e+00,  2.00000000e+00,
         2.00000000e+00,  3.00000000e+00,  4.00000000e+00,
         5.00000000e+00,  6.00000000e+00,  8.